In [1]:
# Define IAM role
import boto3
import re

import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-kmeans-byom"

In [2]:
import numpy as np
import sklearn.cluster
import pickle
import gzip
import urllib.request
import json
import mxnet as mx
import boto3
import time
import io
import os

In [4]:
region = boto3.Session().region_name
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-sample-data-{region}", "algorithms/kmeans/mnist/mnist.pkl.gz", "mnist.pkl.gz"
)
f = gzip.open("mnist.pkl.gz", "rb")
train_set, valid_set, test_set = pickle.load(f, encoding="latin1")
f.close()

In [6]:
kmeans = sklearn.cluster.KMeans(n_clusters=10).fit(train_set[0])

In [8]:
centroids = mx.ndarray.array(kmeans.cluster_centers_)
mx.ndarray.save("model_algo-1", [centroids])

In [9]:
!tar czvf model.tar.gz model_algo-1

model_algo-1


In [10]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "model.tar.gz")
).upload_file("model.tar.gz")

In [11]:
from sagemaker.amazon.amazon_estimator import get_image_uri

kmeans_model = "DEMO-kmeans-byom-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

sm = boto3.client("sagemaker")
container = get_image_uri(boto3.Session().region_name, "kmeans")

create_model_response = sm.create_model(
    ModelName=kmeans_model,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": container,
        "ModelDataUrl": "s3://{}/{}/model.tar.gz".format(bucket, prefix),
    },
)

print(create_model_response["ModelArn"])

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


arn:aws:sagemaker:us-east-1:678287862391:model/demo-kmeans-byom-2022-04-13-10-34-02


In [12]:
kmeans_endpoint_config = "DEMO-kmeans-byom-endpoint-config-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(kmeans_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=kmeans_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": kmeans_model,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-kmeans-byom-endpoint-config-2022-04-13-10-34-26
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:678287862391:endpoint-config/demo-kmeans-byom-endpoint-config-2022-04-13-10-34-26


In [13]:
%%time

kmeans_endpoint = "DEMO-kmeans-byom-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(kmeans_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=kmeans_endpoint, EndpointConfigName=kmeans_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = sm.describe_endpoint(EndpointName=kmeans_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

sm.get_waiter("endpoint_in_service").wait(EndpointName=kmeans_endpoint)

resp = sm.describe_endpoint(EndpointName=kmeans_endpoint)
status = resp["EndpointStatus"]
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation did not succeed")

DEMO-kmeans-byom-endpoint-202204131035
arn:aws:sagemaker:us-east-1:678287862391:endpoint/demo-kmeans-byom-endpoint-202204131035
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:678287862391:endpoint/demo-kmeans-byom-endpoint-202204131035
Status: InService
CPU times: user 148 ms, sys: 18.8 ms, total: 167 ms
Wall time: 5min 1s


In [14]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()

In [15]:
runtime = boto3.Session().client("runtime.sagemaker")

payload = np2csv(train_set[0][0:100])
response = runtime.invoke_endpoint(
    EndpointName=kmeans_endpoint, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
scored_labels = np.array([r["closest_cluster"] for r in result["predictions"]])

In [16]:
scored_labels == kmeans.labels_[0:100]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])